In [ ]:
from __future__ import division
from __future__ import print_function
import win32com.client
import sys
import re
import pandas
import pandas, xgboost, numpy, textblob, string
import pandas as pd
from pandas.compat import StringIO
from langdetect import detect
import io
import os
import unicodecsv as csv
import nltk
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import codecs
import pickle
from spacy.lang.en import English
import spacy
import scipy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from sklearn import feature_extraction
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import seaborn as sns
from math import sqrt
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import mpld3
import uuid
import datetime
from gensim.summarization import keywords
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
rubbish = ["intended recipient", "designated recipient", "consider the environmental", "may contain confidential", 
           "may be confidential", "received this in error", "communication in error", "privileged and confidential",
           "environmental impact", "consider the environment",  "before printing", "Before printing", "Twitter",
           "from an EXTERNAL SENDER", "Sent: ", "Cc: ", "LinkedIn", "Sent from my iPhone", "Thank you in advance.", "RE: ",
          'vriendelijke', 'Euregio Digital Delivery Center', 'information that is confidential', 'for the person or entity']

def import_train_test (output_file, root_folder, folder, year, month, day, hour, minute, second, classes, spamm, rubbish = rubbish):
    importmails(output_file, root_folder, folder, year, month, day, hour, minute, second)
    cleanmails('C:/Users/andrea.feher/Downloads/jk_sct_train.csv', 'en')
    cluster('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete.csv', classes)
    addkeywords ('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete_labeled.csv', classes, spamm)
    trainsvm ('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete_labeled.csv')
    testsvm('C:/Users/andrea.feher/Downloads/jk_sct_test.csv')
    #remove_files('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/')
def train_test (classes, spamm):
    cleanmails('C:/Users/andrea.feher/Downloads/jk_sct_train.csv', 'en')
    cluster('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete.csv', classes)
    addkeywords ('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete_labeled.csv', classes, spamm)
    trainsvm ('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete_labeled.csv')
    testsvm('C:/Users/andrea.feher/Downloads/jk_sct_test.csv')
    #remove_files('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/')
def just_train (classes, spamm):
    cleanmails('C:/Users/andrea.feher/Downloads/jk_sct_train.csv', 'en')
    cluster('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete.csv', classes)
    addkeywords ('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete_labeled.csv', classes, spamm)
    trainsvm ('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete_labeled.csv')
    #remove_files('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/')
def just_test ():
    testsvmm('C:/Users/andrea.feher/Downloads/jk_sct_test1.csv')
    #remove_files('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/')

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")
totalvocab_stemmed = []
totalvocab_tokenized = []


def remove_files (folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)

def importmails(output_file1, root_folder, folder, year, month, day, hour, minute, second):
    output_file = open(output_file1,'wb')    
    output_writer = csv.writer(output_file, delimiter = ",", encoding='latin2', errors = 'ignore')
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    root_folder = outlook.Folders(root_folder)
    folder = root_folder.Folders(folder)
    ssbox = root_folder.Folders[1]
    counter = 0
    for folder in ssbox.Folders:
        counter += 1
    if counter < 2:
        inbox = outlook.GetDefaultFolder(6)
        messages = inbox.Items
        message = messages.GetFirst()
        while message:
            try:
                if message.SenderEmailType=='EX':
                    sender = re.sub(r'.*-', '', message.SenderEmailAddress).lower()+'@accenture.com'
                else:
                    sender = message.SenderEmailAddress
                output_writer.writerow([
                    message.subject,
                    sender,
                    message.Body])
            except Exception as e:
                    ()
            message = messages.GetNext()
        output_file.close()
    else:
        for folder in ssbox.Folders:    
            print(folder)
            messages = folder.Items
            messages = messages.Restrict("[ReceivedTime] >= '" +datetime.datetime(year, month, day, hour, minute, second).strftime('%m/%d/%Y %H:%M %p')+"'")
            message = messages.GetFirst()
            print('-------------')
            print(str(len(list(messages)))+' messages in folder ')
            while message:
                try:
                    if message.SenderEmailType=='EX':
                        sender = re.sub(r'.*-', '', message.SenderEmailAddress).lower()+'@accenture.com'
                    else:
                        sender = message.SenderEmailAddress
                    output_writer.writerow([
                        message.subject,
                        sender,
                        message.Body])
                except Exception as e:
                    ()
                message = messages.GetNext()
        output_file.close()
    split(output_file1)

def split(output_file1):
    df4 = pandas.read_csv(output_file1, header=None, encoding = "latin2")
    train=df4.sample(frac=0.7,random_state=1993)
    test=df4.drop(train.index)
    test2=test.sample(frac=0.99,random_state=1993)
    test3=test.drop(test2.index)
    train.to_csv(r'C:/Users/andrea.feher/Downloads/jk_sct_train.csv',header=None, index=False)
    test2.to_csv(r'C:/Users/andrea.feher/Downloads/jk_sct_test.csv',header=None, index=False)
    test3.to_csv(r'C:/Users/andrea.feher/Downloads/jk_sct_test1.csv',header=None, index=False)
    
def removepersonalnames(text):
    document_big = nlp_big(text)
    entities_big = [e.string for e in document_big.ents if 'PERSON'==e.label_] 
    entities_big = list(entities_big) 
    for i in entities_big:
        if i in text:
            text = text.replace(i, '')
    return text

def cleanmails(output_file, lang):
    try:
        print( )
        print('... Initiating the cleaning process ...')
        df = pandas.read_csv(output_file, header=None, encoding = "latin2")
        sep1 = '< > > wrote:'
        sep2 = ' > het volgende geschreven:'
        oldreplacebody = ('\r', '\t','\n \n','\n\n\n\n','\n\n\n','\n\n')
        newreplacebody = ('','',"\n","\n","\n","\n")
        oldreplacesub = ('[External]', 'FW: ', 'Fw: ', 'Fwd: ', 'RE: ', 'Re: ')
        newreplacesub = ('','','', '','', '')
        for i,j in df.iterrows():
            j[2] = str(j[2])
            j[2] = re.sub('<[^>]+>', '< >', j[2], flags=re.MULTILINE)
            j[2] = re.sub('To:[^>]+\n', '', j[2], flags=re.MULTILINE)
            j[2] = j[2].split(sep1, 1)[0]
            j[2] = j[2].split(sep2, 1)[0]
            for check, rep in zip(oldreplacebody, newreplacebody):
                j[2] = j[2].replace(check, rep)
            j[0] = str(j[0])    
            for check, rep in zip(oldreplacesub, newreplacesub):
                j[0] = j[0].replace(check, rep)
        df.rename(columns={0:'subject', 1:'sender_address', 2: "body"}, inplace=True)
        corpus, ID = [], []
        global cellist
        cellist = []
        for i,j in df.iterrows():
            ID.append(str(uuid.uuid4()))   
        df['ID'] = ID
        for i,j in df.iterrows():
            corpus.append(j['body'])
        length = len(df)
        if len(df) == len(corpus):
            print('The total number of files to clean is ', str(len(df)))
        global nlp_big
        nlp_big =  spacy.load('en_core_web_md')
        if length > 1:
            if length > 1000:
                counter = round(length/50)
            else:
                counter = round(length/10)
            for r in range(0, len(corpus)):
    #            corpus[r] = removepersonalnames(corpus[r])
                convert(corpus[r])
                perc = r * 100/length
                rounded = "{0:.2f}".format(perc)
                if r % counter == 0:
                    print('... Approximately', rounded, ' % of your files are clean, currently cleaning file no. ', len(cellist), '...')
        else:
                counter = round(length/10)
                for r in range(0, len(corpus)):
                    corpus[r] = removepersonalnames(corpus[r])
                    convert(corpus[r])
        df3 = pandas.read_csv('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk2.csv', header=None, encoding = "latin2")
        print('... Filtering emails in English ...')
        df.index = df3.index
        df3['sender_address'] = df['sender_address']
        df3['subject'] = df['subject']
        df3['body'] = df['body']
        df3['ID'] = df['ID']
        df3.rename(columns={0:'corpus'}, inplace=True)
        df3['corpus'] = df3['corpus'].map(str) + '\n' + df3['subject'] + " " + df3['subject'] + " " + df3['subject']
        lang1 = []
        for i,j in df3.iterrows():
            j['corpus'] = str(j['corpus'])
            if len(j['corpus']) > 100:
                l = detect(j['corpus'])
            else:
                l = "email too short"
            lang1.append(l)
        df3['language'] = lang1
        df4 = df3.loc[df3.language == str(lang)]
        df4.to_csv(r'C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete.csv', index=False)
        print ()
        print ('CLEANING COMPLETE')
    except KeyboardInterrupt:
        print('... Interrupted, removing files ...')
        remove_files('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/')

def convert(text, threshold=.95):
    pos_tagger = English()
    original_email = text
    sentences = _corpus2sentences(original_email)
    _generate_text(sentences)

def _corpus2sentences(text):
    return text.strip().split('\n')

def _generate_text(sentences, threshold=0.95):
    sen = '\n'
    clean = ""
    for sentence in sentences: 
        if _prob_block(sentence, nlp_big) < threshold:
            if not any(r in sentence for r in rubbish):
                if clean == '':
                    clean = sen + sentence
                else:
                    clean = clean + sen + sentence
    _generate_list(clean)

def _generate_list(clean):
    cellist.append(clean)
    with open('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk2.csv', 'wb') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for h in cellist:
            wr.writerow([h])
    
def _prob_block(sentence, pos_tagger):   
    doc = pos_tagger(sentence)
    verb_count = np.sum([token.pos_ != "VERB" for token in doc])
    length = float(len(doc))
    thing = 0
    if verb_count > 0:
        thing = verb_count/length
    else:
        thing == 0.0
    return thing

def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

def make_corpus(dataframe):
    corpus = []
    for i,j in dataframe.iterrows():
        corpus.append(j['corpus'])
    return corpus

def make_subject(dataframe):
    subject = []
    for i,j in dataframe.iterrows():
        subject.append(j['subject'])
    return subject

def make_ID(dataframe):
    ID = []
    for i,j in dataframe.iterrows():
        ID.append(j['ID'])
    return ID

def unique(element, listt):
    newlist = []
    for l in listt:
        newlist.append(l)
    return listt.count(element) == 1

def cluster (filename, dictionary):
    try:
        print()
        print('... Clustering into your ' + str(len(dictionary)) + ' topics according to the keywords given ...')
        df = pd.read_csv(filename, encoding = "latin2")
        corpus = make_corpus(df)
        subject = make_subject(df)
        ID = make_ID(df)
        topicc = []
        for j in range(0, len(corpus)):
            corpus[j] = corpus[j].lower()
            readlines = corpus[j].strip().split('\n')
            last = readlines[-1]
            topic = []
            for topics in dictionary.keys():
                topic.append(topics)
                top, top2 = [], []
                for values in dictionary.values():
                    counter = 0
                    for v in values:
                        if values.index(v) <= 2:
                            counter += (last.count(v)) * 3
                    top.append(counter)
            if not all(x == 0 for x in top) and top[0] >= 2:
                topicc.append(topic[0])
            if not all(x == 0 for x in top) and top[0] == 0:
                highest_index = top.index(max(top))
                if unique(max(top), top) == True:
                    topicc.append(topic[highest_index])
                else:
                    for values in dictionary.values():
                        counter = 0
                        for v in values:
                            if values.index(v) == 0:
                                counter += (corpus[j].count(v)) * 3
                            if values.index(v) >= 1 and values.index(v) <= 2:
                                counter += (corpus[j].count(v)) * 2
                            else:
                                counter += corpus[j].count(v)
                        top2.append(counter)
                    print(top2)
                    if all(x == 0 for x in top2) :
                        topicc.append('other')
                    else:
                        highest_index = top2.index(max(top2))
                        if max(top2) > 2:
                            topicc.append(topic[highest_index])
                        else:
                            topicc.append('other')
            if all(x == 0 for x in top):
                for values in dictionary.values():
                    counter = 0
                    for v in values:
                        if values.index(v) == 0:
                            counter += (corpus[j].count(v)) * 3
                        if values.index(v) >= 1 and values.index(v) <= 2:
                            counter += (corpus[j].count(v)) * 2
                        else:
                            counter += corpus[j].count(v)
                    top2.append(counter)
                print(top2)
                if all(x == 0 for x in top2):
                    topicc.append('other')
                else:
                    highest_index = top2.index(max(top2))
                    if max(top2) > 2:
                        topicc.append(topic[highest_index])
                    else:
                        topicc.append('other')
        df['topic'] = topicc
        df.to_csv(r'C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete_labeled.csv', index=False)
        return df
    except KeyboardInterrupt:
        print('... Interrupted, removing files ...')
        remove_files('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/')

def addkeywords (filename, dictionary, spam):
    df = pd.read_csv(filename, encoding = "latin2")
    df['topic'].unique()
    topicss = list(df['topic'].unique())
    counter = 0
    for j in topicss:
        counter += 1
        print( )
        print('... Processing topic', j, '...')
        df1 = df[df.topic == j]
        corpus = make_corpus(df1)
        k = []
        text = ''
        for i in corpus:
            i = i.encode('latin2','replace').lower()
            k.append(i)
        text = b''.join(k)
        values = keywords(text=text,split='\n',scores=True)
        data = pd.DataFrame(values,columns=['keyword','score'])
        data = data.sort_values('score',ascending=False)
        key_word_list = data.head(40)
        keywordlist = list(key_word_list['keyword'])
        for i in keywordlist:
            if i in spam:
                keywordlist.remove(i)
        for i in keywordlist:
            if i in dictionary[j]:
                keywordlist.remove(i)
        dictionary[j].extend(keywordlist)
        print('The recommended keywords for this topic are the following:', dictionary[j])
        
def gettfidf(trainDF, d, count_vect, tfidf_vect):
    print('... Transforming the validation data ...')
    count = count_vect.transform(d)
    tfidf_vect.fit(trainDF['corpus'])
    tfidf = tfidf_vect.transform(d)
    return tfidf   

def gettfidf2(trainDF, d, count_vect, tfidf_vect):
    print('... Transforming the training data ...')
    count = count_vect.transform(d)
    tfidf_vect.fit(trainDF['corpus'])
    tfidf1 = tfidf_vect.transform(d)
    return tfidf1

def tdf (data):
    labels, texts, ID = [], [], []
    for i, line in data.iterrows():
        labels.append(line['topic'])
        texts.append(line['corpus'])
        ID.append(line['ID'])
    print('... Creating a dataframe using just texts and labels ...')
    trainDF = pandas.DataFrame()
    trainDF['corpus'] = texts
    trainDF['topic'] = labels
    trainDF['ID'] = ID
    return trainDF

def trainsvm (file):
    try:
        print( )
        print('... Processing the training set ...')
        data = pandas.read_csv(file, encoding = "latin2")
        trainDF = tdf (data)
        train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['corpus'], trainDF['topic'], 
                                                                              test_size=0.2, random_state=1993)
        topicsinwords = list(trainDF['topic'].unique())
        topicsinwords = sorted(topicsinwords)
        print('... Encoding the topic number ...')
        encoder = preprocessing.LabelEncoder()
        train_y = encoder.fit_transform(train_y)
        valid_y = encoder.fit_transform(valid_y)
        topicsinnumbers = list(set(list(train_y)))
        count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', decode_error="replace")
        xtrain_count = count_vect.fit_transform(train_x)
        pickle.dump(count_vect.vocabulary_,open("C:/Users/andrea.feher/Downloads/feature.pkl","wb"))
        xvalid_count = count_vect.transform(valid_x)
        tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=1000)
        tfidf_vect.fit(trainDF['corpus'])
        pickle.dump(tfidf_vect.vocabulary_,open("C:/Users/andrea.feher/Downloads/features.pkl","wb"))
        xtrain_tfidf = gettfidf(trainDF, train_x, count_vect, tfidf_vect)
        xvalid_tfidf = gettfidf2(trainDF, valid_x, count_vect, tfidf_vect)
        print('... Fitting the training dataset on the classifier ...')
        Cs = [0.01, 0.1, 1, 10, 100, 1000, 10000]
        max_iter = [1, 10, 1000, 100, 10000]
        param_grid = {'C': Cs, 'max_iter' : max_iter}
        grid_search = GridSearchCV(svm.LinearSVC(), param_grid, cv = 3)
        print('... Grid ...')
        grid_search.fit(xtrain_tfidf, train_y)
        C = grid_search.best_params_['C']
        iterations = grid_search.best_params_['max_iter']
        print('C =', C, ', Iterations =', iterations)
        model = LinearSVC(C=C, max_iter = iterations)
        model.fit(xtrain_tfidf, train_y)
        valid_predictions_SVM = model.predict(xvalid_tfidf)
        print("SVM Accuracy Score -> ",accuracy_score(valid_predictions_SVM, valid_y)*100)
        check = pandas.DataFrame()
        check['pred'] = valid_predictions_SVM
        check['actual'] = valid_y
        check['Index'] = valid_x.index
        check.index = valid_x.index
        p = list(check.index)
        rowData = data.iloc[p, : ]
        rowData.index = check.index
        rowData['pred'] = check['pred']
        rowData['pred'] = encoder.inverse_transform(rowData['pred'])
        print(rowData[['subject', 'topic', 'pred']].head(30))
        print("... Saving the tfidf dataframe ...")
        tfidfdf = pandas.DataFrame(xtrain_tfidf.toarray())
        tfidfdf['label'] = train_y
        tfidfdf.to_csv(r'C:/Users/andrea.feher/Downloads/tfidf.csv', encoding = "latin2")
        labelencoding = pandas.DataFrame()
        labelencoding['topic_number'] = topicsinnumbers
        labelencoding['topic_name'] = topicsinwords
        labelencoding.to_csv(r'C:/Users/andrea.feher/Downloads/labeling.csv', encoding = "latin2")
    except KeyboardInterrupt:
        print('... Interrupted, removing files ...')
        remove_files('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/')

def calculate_wcss(data):
    wcss = []
    for n in range(2, 21):
        kmeans = KMeans(n_clusters=n)
        kmeans.fit(X=data)
        wcss.append(kmeans.inertia_)
    return wcss

def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss)-1]
    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    return distances.index(max(distances)) + 2
        
def testsvm (output_file):
    try:
        print( )
        print('... Initiating testing ...')
        cleanmails (output_file, 'en')
        df7 = pandas.read_csv('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete.csv', encoding = "latin2")
        df8 = pandas.read_csv('C:/Users/andrea.feher/Downloads/labeling.csv', encoding = "latin2")
        tfidfdf = pandas.read_csv(r'C:/Users/andrea.feher/Downloads/tfidf.csv', encoding = "latin2")
        tfidfdf = tfidfdf.dropna()
        train_y = tfidfdf['label']
        xtrain_tfidf = tfidfdf.drop(['label'], axis=1)
        xtrain_tfidf = xtrain_tfidf.drop(['0'], axis=1)
        xtrain_tfidf = scipy.sparse.csr_matrix(xtrain_tfidf)
        labels, textsss, labelno, subject, ID = [], [], [], [], []
        textsss = make_corpus(df7)
        subject = make_subject(df7)
        ID = make_ID(df7)
        for i, line in df8.iterrows():
            labels.append(line['topic_name'])
        for i, line in df8.iterrows():
            labelno.append(line['topic_number'])
        testDF = pandas.DataFrame()
        testDF['corpus'] = textsss
        for n in range(0, len(textsss)):
            if type(textsss[n]) == str:
                allwords_stemmed = tokenize_and_stem(textsss[n])
                totalvocab_stemmed.extend(allwords_stemmed)
                allwords_tokenized = tokenize_only(textsss[n])
                totalvocab_tokenized.extend(allwords_tokenized)
        vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
        print('... Vectorizing ...')
        tf1 = open("C:/Users/andrea.feher/Downloads/feature.pkl", 'rb')
        tf2 = open("C:/Users/andrea.feher/Downloads/features.pkl", 'rb')
        count_vect = CountVectorizer(decode_error="replace", max_features=1000, vocabulary=pickle.load(tf1))
        xtest_count = count_vect.fit_transform(testDF['corpus'])
        #pickle.dump(count_vect.vocabulary_,open("C:/Users/andrea.feher/Downloads/feature.pkl","ab"))
        pickle.dump(count_vect.vocabulary_,open("C:/Users/andrea.feher/Downloads/feature.pkl","wb"))
        tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', decode_error="replace",
                max_features=1000, tokenizer=tokenize_and_stem, stop_words='english', ngram_range=(1,3),
                use_idf=True, vocabulary=pickle.load(tf2))
        xtest_tfidf = tfidf_vect.fit(testDF['corpus'])
        xtest_tfidf = tfidf_vect.transform(testDF['corpus'])
        #pickle.dump(tfidf_vect.vocabulary_,open("C:/Users/andrea.feher/Downloads/features.pkl","ab"))
        pickle.dump(tfidf_vect.vocabulary_,open("C:/Users/andrea.feher/Downloads/features.pkl","wb"))
        print('This is the shape of your tfidf matrix: ', xtest_tfidf.shape, xtrain_tfidf.shape)
        terms = tfidf_vect.get_feature_names()
        dist = 1 - cosine_similarity(xtest_tfidf)
        print()
        sum_of_squares = calculate_wcss(xtest_tfidf)
        print('... Calculating the recommended number of topics in the dataset ...')
        num_clusters = optimal_number_of_clusters(sum_of_squares)
        print('According to the calculation, you have ', str(num_clusters), ' topics.')
        print('... Grouping documents into ', str(num_clusters), 'clusters... ')
        if len(textsss) > 500:
            print('... This might take a while, as ', (len(textsss)), 'documents are being read ... ')
        km = KMeans(n_clusters=num_clusters, random_state=3)
        %time km.fit(xtest_tfidf)
        clusters = km.labels_.tolist()
        joblib.dump(km,  'doc_cluster.pkl')
        km = joblib.load('doc_cluster.pkl')
        clusters = km.labels_.tolist()
        files = { 'title': subject, 'cluster': clusters, 'ID': ID}
        frame = pd.DataFrame(files, index = [clusters] , columns = ['title', 'cluster', 'ID'])
        frame['cluster'].value_counts()
        print("Top terms per cluster:")
        print()
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]
        for i in range(num_clusters):
            print()
            print("Cluster %d words:" % i, end='')
            for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
                print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
            print()
            print("Files belonging to Cluster %d:" % i, end='')
            for title in frame.ix[i]['title'].values.tolist():
                print(' %s,' % title, end='')
            print()
        print()
        Cs = [0.1, 1, 10, 100, 1000, 10000]
        max_iter = [10, 100, 1000, 10000]
        param_grid = {'C': Cs, 'max_iter' : max_iter}
        grid_search = GridSearchCV(svm.LinearSVC(), param_grid, cv = 3)
        print('... Grid ...')
        grid_search.fit(xtrain_tfidf, train_y)
        C = grid_search.best_params_['C']
        iterations = grid_search.best_params_['max_iter']
        print('C =', C, ', Iterations =', iterations)
        print('... Support Vector Machine doing its magic ...')
        model = LinearSVC(C=C, max_iter = iterations, class_weight = 'balanced')
        model.fit(xtrain_tfidf, train_y)
        test_predictions_SVM = model.predict(xtest_tfidf)
        check = pandas.DataFrame()
        check['pred'] = test_predictions_SVM
        rowData = df7
        rowData.index = check.index
        rowData['pred'] = check['pred']
        for i in range(len(labels)):
            rowData['pred'] = rowData['pred'].replace({labelno[i]:labels[i]})
        rowData.to_csv(r'C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/tested-labeled.csv', index=False)
        print('File saved!')
    except KeyboardInterrupt:
        print('... Interrupted, removing files ...')
        remove_files('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/')

def testsvmm (output_file):
    try:
        print('... Classification initiated ...')
        cleanmails (output_file, 'en')
        df7 = pandas.read_csv('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/jk_sct_eng_complete.csv', encoding = "latin2")
        df8 = pandas.read_csv('C:/Users/andrea.feher/Downloads/labeling.csv', encoding = "latin2")
        tfidfdf = pandas.read_csv(r'C:/Users/andrea.feher/Downloads/tfidf.csv', encoding = "latin2")
        tfidfdf = tfidfdf.dropna()
        train_y = tfidfdf['label']
        xtrain_tfidf = tfidfdf.drop(['label'], axis=1)
        xtrain_tfidf = xtrain_tfidf.drop(['0'], axis=1)
        xtrain_tfidf = scipy.sparse.csr_matrix(xtrain_tfidf)
        labels, textsss, labelno, subject, ID = [], [], [], [], []
        textsss = make_corpus(df7)
        subject = make_subject(df7)
        ID = make_ID(df7)
        for i, line in df8.iterrows():
            labels.append(line['topic_name'])
        for i, line in df8.iterrows():
            labelno.append(line['topic_number'])
        testDF = pandas.DataFrame()
        testDF['corpus'] = textsss
        for n in range(0, len(textsss)):
            if type(textsss[n]) == str:
                allwords_stemmed = tokenize_and_stem(textsss[n])
                totalvocab_stemmed.extend(allwords_stemmed)
                allwords_tokenized = tokenize_only(textsss[n])
                totalvocab_tokenized.extend(allwords_tokenized)
        vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
        print('... Vectorizing ...')
        tf1 = open("C:/Users/andrea.feher/Downloads/feature.pkl", 'rb')
        tf2 = open("C:/Users/andrea.feher/Downloads/features.pkl", 'rb')
        count_vect = CountVectorizer(decode_error="replace", max_features=1000, vocabulary=pickle.load(tf1))
        xtest_count = count_vect.fit_transform(testDF['corpus'])
        #pickle.dump(count_vect.vocabulary_,open("C:/Users/andrea.feher/Downloads/feature.pkl","ab"))
        pickle.dump(count_vect.vocabulary_,open("C:/Users/andrea.feher/Downloads/feature.pkl","wb"))
        tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', decode_error="replace",
                max_features=1000, tokenizer=tokenize_and_stem, stop_words='english', ngram_range=(1,3),
                use_idf=True, vocabulary=pickle.load(tf2))
        xtest_tfidf = tfidf_vect.fit(testDF['corpus'])
        xtest_tfidf = tfidf_vect.transform(testDF['corpus'])
        #pickle.dump(tfidf_vect.vocabulary_,open("C:/Users/andrea.feher/Downloads/features.pkl","ab"))
        pickle.dump(tfidf_vect.vocabulary_,open("C:/Users/andrea.feher/Downloads/features.pkl","wb"))
        print('This is the shape of your tfidf matrix: ', xtest_tfidf.shape, xtrain_tfidf.shape)
        Cs = [0.1, 1, 10, 100, 1000, 10000]
        max_iter = [10, 100, 1000, 10000]
        param_grid = {'C': Cs, 'max_iter' : max_iter}
        grid_search = GridSearchCV(svm.LinearSVC(), param_grid, cv = 3)
        print('... Grid ...')
        grid_search.fit(xtrain_tfidf, train_y)
        C = grid_search.best_params_['C']
        iterations = grid_search.best_params_['max_iter']
        print('C =', C, ', Iterations =', iterations)
        print('... Support Vector Machine doing its magic ...')
        model = LinearSVC(C=C, max_iter = iterations, class_weight = 'balanced')
        model.fit(xtrain_tfidf, train_y)
        test_predictions_SVM = model.predict(xtest_tfidf)
        check = pandas.DataFrame()
        check['pred'] = test_predictions_SVM
        rowData = df7
        rowData.index = check.index
        rowData['pred'] = check['pred']
        for i in range(len(labels)):
            rowData['pred'] = rowData['pred'].replace({labelno[i]:labels[i]})
        rowData.to_csv(r'C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/tested-labeled.csv', index=False)
        print('File saved!')
    except KeyboardInterrupt:
        print('... Interrupted, removing files ...')
        remove_files('C:/Users/andrea.feher/Downloads/Python Notebooks and Scripts/emails/')

In [ ]:
import_train_test('C:/Users/andrea.feher/Downloads/jk_sct.csv', 'support_robotics@accenture.com', "Postvak IN", 
                  2018, 9, 1, 0, 0, 0, 
           {'urgent' : ['p1', 'urgent', 'important', 'asap', 'high priority', 'priority','p2', 
                       'as soon as possible', 'stopped', 'priority 1', 'prompt', 'urgent:', 'important:'],
           'incident/change tickets' : ['description: ', 'assigned to', 'automatically generated', 'incident', 
                        'change', 'assigned', 'on your behalf', 'click here'], 
           'credentials' : ['password', 'new', 'reset', 'maintenance', 'credential', 'login', 'log in', 
                        'credentials', 'expire', 'user'], 
            'pop-up' : ['pop-up', 'popup', 'pop up', 'popping', 'popped', 'block', 'message', 'window'], 
             'infrastructure' : ['connection', 'sap', 'connectivity', 'virtual machine', 'item', 'reboot', 
                        'output', 'inform', 'fail', 'failing', 'infrastructure'], 
           'meetings' : ['call', 'meet', 'event', 'meeting', 'meetings', 'call', 'calls', 'join', 'joining', 
                         'skype', 'teams', 'schedule', 'tomorrow'], 
           'reports' : ['report', 'daily', 'snow', 'performance', 'process', 'service-now', 'complete', 
                        'reports', 'servicenow', 'service now', 'status'],
           'SLA_warning' : ['sla', 'warning', 'breach', 'task state', 'robotics', 'task'], 
           'documentation' : ['pdd', 'tdd', 'sdd', 'share','documentation', 'fdd', 'change', 'file', 'manual', 
                        'brd',  'update', 'review', 'design document'], 
           'lifecycle' : ['handover', 'lifecycle', 'go-live', 'go live', 'test', 'testing', 'knowledge transfer', 
                          'hand-over'],
           'other' : ['out of office','office']
          }, ['robotics', 'issue', 'run', 'runs', 'flight', 'flights', 'running', 'case', 'case', 'robots', 'errors', 
         'rpa', 'nice', 'process', 'processing', 'processes', 'processed', 'email', 'emails', 'thank', 'thanks'])


In [ ]:
train_test( {'urgent' : ['p1', 'urgent', 'important', 'asap', 'high priority', 'priority','p2', 
                       'as soon as possible', 'stopped', 'priority 1', 'prompt', 'urgent:', 'important:'],
           'incident/change tickets' : ['description: ', 'assigned to', 'automatically generated', 'incident', 
                        'change', 'assigned', 'on your behalf', 'click here'], 
           'credentials' : ['password', 'new', 'reset', 'maintenance', 'credential', 'login', 'log in', 
                        'credentials', 'expire', 'user'], 
            'pop-up' : ['pop-up', 'popup', 'pop up', 'popping', 'popped', 'block', 'message', 'window'], 
             'infrastructure' : ['connection', 'sap', 'connectivity', 'virtual machine', 'item', 'reboot', 
                        'output', 'inform', 'fail', 'failing', 'infrastructure'], 
           'meetings' : ['call', 'meet', 'event', 'meeting', 'meetings', 'call', 'calls', 'join', 'joining', 
                         'skype', 'teams', 'schedule', 'tomorrow'], 
           'reports' : ['report', 'daily', 'snow', 'performance', 'process', 'service-now', 'complete', 
                        'reports', 'servicenow', 'service now', 'status'],
           'SLA_warning' : ['sla', 'warning', 'breach', 'task state', 'robotics', 'task'], 
           'documentation' : ['pdd', 'tdd', 'sdd', 'share','documentation', 'fdd', 'change', 'file', 'manual', 
                        'brd',  'update', 'review', 'design document'], 
           'lifecycle' : ['handover', 'lifecycle', 'go-live', 'go live', 'test', 'testing', 'knowledge transfer', 
                          'hand-over'],
           'other' : ['out of office','office']
          }, ['robotics', 'issue', 'run', 'runs', 'flight', 'flights', 'running', 'case', 'case', 'robots', 'errors', 
         'rpa', 'nice', 'process', 'processing', 'processes', 'processed', 'email', 'emails', 'thank', 'thanks'])


In [ ]:
just_test()

In [ ]:
train_test({'kahoots' : ['kahoot', 'quiz', 'answers', 'questions', 'correct', 'account'], 
    'courses/learning boards' : ['learning board', 'course', 'training', 'exam', 'program', 'register', 'skill', 'completed', 'transcript', 'curriculum', 'accreditation'],  
    'meetings' : ['meeting', 'canceled', 'event', 'cancelled', 'meeting', 'attend', 'rpa team', 'lunch', 'join', 'session', 'party'], 
    'myTE' : ['myte', 'wbs', 'roll', 'myt&e', 'submit', 'submission', 'hours', 'unassigned', 'chargeable'], 
    'messages' : ['skype', 'microsoft teams', 'message', 'join', 'conversation', 'video'],
    'accenture spam': ['newsletter', 'action', 'gallia', 'feedback', 'take action', 'habit', 'conversant', 'trending', 'award', 'versed', 'tech'],
    'other' : ["password"]},
    ['email', 'emails', 'accenture', 'thank', 'thanks'])

In [ ]:
import_train_test('C:/Users/andrea.feher/Downloads/jk_sct.csv', 'andrea.feher@accenture.com', "Inbox", 
    2019, 5, 30, 0, 0, 0,
    {'kahoots' : ['kahoot', 'quiz', 'answers', 'questions', 'correct', 'account'], 
    'courses/learning boards' : ['learning board', 'course', 'training', 'exam', 'program', 'register', 'skill', 'completed', 'transcript', 'curriculum', 'accreditation'],  
    'meetings' : ['meeting', 'event', 'attend', 'rpa team', 'lunch', 'join', 'session', 'party'], 
    'myTE' : ['myte', 'wbs', 'roll', 'myt&e', 'submit', 'submission', 'hours', 'unassigned', 'chargeable'], 
    'messages' : ['skype', 'microsoft teams', 'video', 'join', 'conversation', 'message'],
    'accenture spam': ['action', 'newsletter', 'gallia', 'feedback', 'take action', 'habit', 'conversant', 'trending', 'award', 'versed', 'tech'],
    'other' : ["password"]},
    ['email', 'emails', 'accenture', 'thank', 'thanks'])


In [ ]:
testsvmm('C:/Users/andrea.feher/Downloads/jk_sct_test1.csv')